In [47]:
import tensorflow as tf
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.datasets import fetch_20newsgroups
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import euclidean_distances

In [4]:
print("Fecthing data..................................")
newsgroups_train = fetch_20newsgroups(subset='train')
newsgroups_test = fetch_20newsgroups(subset='test')
newsgroups_data = newsgroups_train.data + newsgroups_test.data
newsgroups_labels = list(newsgroups_train.target) + list(newsgroups_test.target)


Fecthing data..................................


In [10]:
print(newsgroups_train.target_names[1])
print(newsgroups_train.target_names[16])

comp.graphics
talk.politics.guns


In [12]:
newsgroups_data_downsampled = []
newsgroups_labels_downsampled = []
for index in range(len(newsgroups_data)):
    if newsgroups_labels[index] == 1 or newsgroups_labels[index] == 16:
        newsgroups_data_downsampled.append(newsgroups_data[index])
        newsgroups_labels_downsampled.append(newsgroups_labels[index])

In [14]:
len(newsgroups_labels_downsampled)

1883

In [15]:
print("Data fetched successfully")
print("Turning data into Tf-IDF format................")
vectorizer = TfidfVectorizer(stop_words='english')
vectors = vectorizer.fit_transform(newsgroups_data_downsampled)

print("Successfully created bag of words.")

Data fetched successfully
Turning data into Tf-IDF format................
Successfully created bag of words.


In [18]:
test_data = vectors[int(0.9 * vectors.shape[0]):]
train_data = vectors[:int(0.9 * vectors.shape[0])]

In [20]:
train_data.shape

(1694, 31911)

In [29]:
learning_rate = 0.001
num_steps = 1800
batch_size = 2

display_step = 100
examples_to_show = 10

# Network Parameters
num_hidden_1 = 1000 # 1st layer num features
# num_hidden_2 = 300 # 2nd layer num features (the latent dim)
num_input = train_data.shape[1] # MNIST data input (img shape: 28*28)

# tf Graph input (only pictures)
X = tf.placeholder("float", [None, num_input])

weights = {
    'encoder_h1': tf.Variable(tf.random_normal([num_input, num_hidden_1])),
    'decoder_h1': tf.Variable(tf.random_normal([num_hidden_1, num_input])),
}
biases = {
    'encoder_b1': tf.Variable(tf.random_normal([num_hidden_1])),
    'decoder_b1': tf.Variable(tf.random_normal([num_input])),
}

In [30]:
def encoder(x):
    # Encoder Hidden layer with sigmoid activation #1
    layer_1 = tf.nn.sigmoid(tf.add(tf.matmul(x, weights['encoder_h1']),
                                   biases['encoder_b1']))
    return layer_1


# Building the decoder
def decoder(x):
    # Decoder Hidden layer with sigmoid activation #1
    layer_1 = tf.nn.sigmoid(tf.add(tf.matmul(x, weights['decoder_h1']),
                                   biases['decoder_b1']))
    return layer_1

In [31]:
# Construct model
encoder_op = encoder(X)
decoder_op = decoder(encoder_op)
# Prediction
y_pred = decoder_op
# Targets (Labels) are the input data.
y_true = X

# Define loss and optimizer, minimize the squared error
loss = tf.reduce_mean(tf.pow(y_true - y_pred, 2))
optimizer = tf.train.RMSPropOptimizer(learning_rate).minimize(loss)

# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()

In [32]:
vec = np.array(vectors.todense())

In [33]:
vec.shape

(1883, 31911)

In [34]:
sess = tf.Session()
sess.run(init)

# Training
for index in range(850):
    data = vec[index * batch_size: (index + 1) * batch_size]
#     data = data.todense()
#     print()
    # Run optimization op (backprop) and cost op (to get loss value)
    _, l = sess.run([optimizer, loss], feed_dict={X: data})
    # Display logs per step
#     if index % display_step == 0 or index == 1:
    print('Step %i: Minibatch Loss: %f' % (index, l))

Step 0: Minibatch Loss: 0.474970
Step 1: Minibatch Loss: 0.476070
Step 2: Minibatch Loss: 0.476575
Step 3: Minibatch Loss: 0.475711
Step 4: Minibatch Loss: 0.477324
Step 5: Minibatch Loss: 0.474016
Step 6: Minibatch Loss: 0.476100
Step 7: Minibatch Loss: 0.478570
Step 8: Minibatch Loss: 0.477533
Step 9: Minibatch Loss: 0.477592
Step 10: Minibatch Loss: 0.477416
Step 11: Minibatch Loss: 0.475954
Step 12: Minibatch Loss: 0.477439
Step 13: Minibatch Loss: 0.478193
Step 14: Minibatch Loss: 0.475538
Step 15: Minibatch Loss: 0.475331
Step 16: Minibatch Loss: 0.474821
Step 17: Minibatch Loss: 0.476503
Step 18: Minibatch Loss: 0.477818
Step 19: Minibatch Loss: 0.476572
Step 20: Minibatch Loss: 0.478039
Step 21: Minibatch Loss: 0.476497
Step 22: Minibatch Loss: 0.478013
Step 23: Minibatch Loss: 0.477858
Step 24: Minibatch Loss: 0.476180
Step 25: Minibatch Loss: 0.477518
Step 26: Minibatch Loss: 0.476573
Step 27: Minibatch Loss: 0.476764
Step 28: Minibatch Loss: 0.476145
Step 29: Minibatch Loss:

Step 238: Minibatch Loss: 0.445325
Step 239: Minibatch Loss: 0.444930
Step 240: Minibatch Loss: 0.444125
Step 241: Minibatch Loss: 0.440234
Step 242: Minibatch Loss: 0.440234
Step 243: Minibatch Loss: 0.440279
Step 244: Minibatch Loss: 0.438154
Step 245: Minibatch Loss: 0.438886
Step 246: Minibatch Loss: 0.438711
Step 247: Minibatch Loss: 0.434337
Step 248: Minibatch Loss: 0.437322
Step 249: Minibatch Loss: 0.435238
Step 250: Minibatch Loss: 0.433332
Step 251: Minibatch Loss: 0.430027
Step 252: Minibatch Loss: 0.432478
Step 253: Minibatch Loss: 0.430273
Step 254: Minibatch Loss: 0.430560
Step 255: Minibatch Loss: 0.429276
Step 256: Minibatch Loss: 0.429663
Step 257: Minibatch Loss: 0.427857
Step 258: Minibatch Loss: 0.426861
Step 259: Minibatch Loss: 0.426115
Step 260: Minibatch Loss: 0.424281
Step 261: Minibatch Loss: 0.423346
Step 262: Minibatch Loss: 0.422558
Step 263: Minibatch Loss: 0.420988
Step 264: Minibatch Loss: 0.420846
Step 265: Minibatch Loss: 0.421032
Step 266: Minibatch 

Step 473: Minibatch Loss: 0.289550
Step 474: Minibatch Loss: 0.288015
Step 475: Minibatch Loss: 0.289864
Step 476: Minibatch Loss: 0.289087
Step 477: Minibatch Loss: 0.287014
Step 478: Minibatch Loss: 0.285217
Step 479: Minibatch Loss: 0.284521
Step 480: Minibatch Loss: 0.285172
Step 481: Minibatch Loss: 0.285681
Step 482: Minibatch Loss: 0.284947
Step 483: Minibatch Loss: 0.286479
Step 484: Minibatch Loss: 0.285777
Step 485: Minibatch Loss: 0.283862
Step 486: Minibatch Loss: 0.285049
Step 487: Minibatch Loss: 0.284359
Step 488: Minibatch Loss: 0.282706
Step 489: Minibatch Loss: 0.282288
Step 490: Minibatch Loss: 0.281327
Step 491: Minibatch Loss: 0.281429
Step 492: Minibatch Loss: 0.285188
Step 493: Minibatch Loss: 0.280188
Step 494: Minibatch Loss: 0.279307
Step 495: Minibatch Loss: 0.280821
Step 496: Minibatch Loss: 0.280460
Step 497: Minibatch Loss: 0.280950
Step 498: Minibatch Loss: 0.281584
Step 499: Minibatch Loss: 0.278961
Step 500: Minibatch Loss: 0.275918
Step 501: Minibatch 

Step 708: Minibatch Loss: 0.227593
Step 709: Minibatch Loss: 0.228099
Step 710: Minibatch Loss: 0.228308
Step 711: Minibatch Loss: 0.226424
Step 712: Minibatch Loss: 0.227437
Step 713: Minibatch Loss: 0.226786
Step 714: Minibatch Loss: 0.226697
Step 715: Minibatch Loss: 0.228257
Step 716: Minibatch Loss: 0.226096
Step 717: Minibatch Loss: 0.226630
Step 718: Minibatch Loss: 0.226641
Step 719: Minibatch Loss: 0.225598
Step 720: Minibatch Loss: 0.225656
Step 721: Minibatch Loss: 0.227118
Step 722: Minibatch Loss: 0.225311
Step 723: Minibatch Loss: 0.224506
Step 724: Minibatch Loss: 0.225464
Step 725: Minibatch Loss: 0.225126
Step 726: Minibatch Loss: 0.223403
Step 727: Minibatch Loss: 0.224429
Step 728: Minibatch Loss: 0.225543
Step 729: Minibatch Loss: 0.224182
Step 730: Minibatch Loss: 0.224638
Step 731: Minibatch Loss: 0.223829
Step 732: Minibatch Loss: 0.224685
Step 733: Minibatch Loss: 0.223051
Step 734: Minibatch Loss: 0.224341
Step 735: Minibatch Loss: 0.224895
Step 736: Minibatch 

In [35]:
reconstructed_data = []
g = sess.run(decoder_op, feed_dict={X:vec})
reconstructed_data = list(g)

In [44]:
reconstructed_data_reduced_dim = []
g = sess.run(encoder_op, feed_dict={X:vec})
reconstructed_data_reduced_dim = list(g)

<b> Accuracy of original data </b>

In [43]:
from sklearn import linear_model
from sklearn.metrics import accuracy_score
log_reg = linear_model.LogisticRegression()
log_reg.fit(vec[:int(0.9 * len(reconstructed_data)) ], newsgroups_labels_downsampled[:int(0.9 * len(reconstructed_data))])
predicted_labels = log_reg.predict(vec[int(0.9 * len(reconstructed_data)):])
score = accuracy_score(newsgroups_labels_downsampled[int(0.9 * len(reconstructed_data)):],predicted_labels)
print("Accuracy of predictions: " ,score * 100, "%")

Accuracy of predictions:  97.88359788359789 %


<b> Accuracy of Reconstructed data </b>

In [40]:
from sklearn import linear_model
from sklearn.metrics import accuracy_score
log_reg = linear_model.LogisticRegression()
log_reg.fit(reconstructed_data[:int(0.9 * len(reconstructed_data)) ], newsgroups_labels_downsampled[:int(0.9 * len(reconstructed_data))])
predicted_labels = log_reg.predict(reconstructed_data[int(0.9 * len(reconstructed_data)):])
score = accuracy_score(newsgroups_labels_downsampled[int(0.9 * len(reconstructed_data)):],predicted_labels)
print("Accuracy of predictions: " ,score * 100, "%")

Accuracy of predictions:  94.70899470899471 %


<b>Accuracy of data in reduced dimensions(1000)</b>

In [46]:
from sklearn import linear_model
from sklearn.metrics import accuracy_score
log_reg = linear_model.LogisticRegression()
log_reg.fit(reconstructed_data_reduced_dim[:int(0.9 * len(reconstructed_data_reduced_dim)) ], newsgroups_labels_downsampled[:int(0.9 * len(reconstructed_data))])
predicted_labels = log_reg.predict(reconstructed_data_reduced_dim[int(0.9 * len(reconstructed_data)):])
score = accuracy_score(newsgroups_labels_downsampled[int(0.9 * len(reconstructed_data)):],predicted_labels)
print("Accuracy of predictions: " ,score * 100, "%")

Accuracy of predictions:  97.35449735449735 %


In [48]:
distances_original_data = euclidean_distances(vec)
distances_reconstructed_data = euclidean_distances(reconstructed_data)
distances_reduced_dim_data = euclidean_distances(reconstructed_data_reduced_dim)

In [49]:
print(distances_original_data.shape)
print(distances_reconstructed_data.shape)
print(distances_reduced_dim_data.shape)

(1883, 1883)
(1883, 1883)
(1883, 1883)


In [56]:
test_index = 20
test_data_distances = distances_original_data[test_index]
test_recons_ditances = distances_reconstructed_data[test_index]
test_data_reduced_distances = distances_reconstructed_data[test_index]
original_nearest_neighs = sorted(range(len(test_data_distances)), key=lambda i: test_data_distances[i], reverse=True)[:20]
recons_nearest_neighs = sorted(range(len(test_recons_ditances)), key=lambda i: test_recons_ditances[i], reverse=True)[:20]
reduced_nearest_neighs = sorted(range(len(test_data_reduced_distances)), key=lambda i: test_data_reduced_distances[i], reverse=True)[:20]
print(original_nearest_neighs)
print(recons_nearest_neighs)
print(reduced_nearest_neighs)

[341, 239, 704, 1784, 1680, 254, 1060, 1219, 836, 39, 235, 1112, 1066, 1837, 307, 311, 746, 607, 433, 521]
[1777, 985, 1700, 1041, 1290, 1701, 497, 1183, 953, 1112, 311, 743, 1066, 622, 652, 727, 355, 692, 514, 631]
[1777, 985, 1700, 1041, 1290, 1701, 497, 1183, 953, 1112, 311, 743, 1066, 622, 652, 727, 355, 692, 514, 631]
